# Elhub API data - Gridloss - Summerproject 2025

## Merging data from Elhub API

Bjørn Eirik Rognskog Nordbak

### Importing data from Elhub API
https://api.elhub.no/energy-data-api#/grid-areas

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

oslo = ZoneInfo("Europe/Oslo")

def fetch_window(start_dt, end_dt):
    params = {
        "dataset":   "LOSS_PER_MGA_HOUR",
        "startDate": start_dt.isoformat(),
        "endDate":   end_dt.isoformat(),
    }
    url = "https://api.elhub.no/energy-data/v0/grid-areas"
    resp = requests.get(url, params=params)
    obj = resp.json()
    
    # --- safeguard: if there's no "data", bail with empty DF ----
    raw = obj.get("data")
    if raw is None:
        print(f"  → no 'data' for {start_dt.date()} → {end_dt.date()}, skipping")
        return pd.DataFrame()
    
    # otherwise flatten
    df = pd.json_normalize(
        raw,
        record_path=["attributes", "lossPerMgaHour"],
        meta=[
            ["attributes", "eic"],
            ["attributes", "name"],
            ["attributes", "status"],
        ],
        errors="ignore"
    ).rename(columns={
        "attributes.eic":    "eic",
        "attributes.name":   "name",
        "attributes.status": "status",
    })
    return df

# loop as before
span_start = datetime(2023,1,1,0,0, tzinfo=oslo)
span_end   = datetime(2025,6,1,0,0, tzinfo=oslo)
window = timedelta(days=7)

all_chunks = []
cur = span_start
while cur < span_end:
    nxt = min(cur + window, span_end)
    print(f"Fetching {cur.date()} → {nxt.date()}")
    dfc = fetch_window(cur, nxt)
    all_chunks.append(dfc)
    cur = nxt

big_df = pd.concat(all_chunks, ignore_index=True)


### Save the data to a CSV file (optional)

In [ ]:
import pandas as pd

# 1. Save your DataFrame to CSV
# Replace big_df with your DataFrame variable
big_df.to_csv('big_df.csv', index=False)

### Reading data from premade CSV file (if neede)

In [3]:
import pandas as pd

big_df = pd.read_csv('big_df.csv')

big_df.head()

,calculatedLossFraction,calculatedLossQuantityKwh,endTime,gridArea,lastUpdatedTime,netInfeedQuantityKwh,priceArea,startTime,eic,name,status
0,0.062064,1451.970,2023-01-01T01:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23394.820,NO2,2023-01-01T00:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
1,0.059934,1399.627,2023-01-01T02:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23352.914,NO2,2023-01-01T01:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
2,0.061977,1430.195,2023-01-01T03:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23076.060,NO2,2023-01-01T02:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
3,0.060450,1373.440,2023-01-01T04:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,22720.318,NO2,2023-01-01T03:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active
4,0.060322,1360.366,2023-01-01T05:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,22551.768,NO2,2023-01-01T04:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active


### Importing data from Elhub API
https://api.elhub.no/energy-data/v0/parties

In [4]:
import requests
import pandas as pd

# 1) Fetch the JSON
url = "https://api.elhub.no/energy-data/v0/parties"
resp = requests.get(url, headers={"Accept": "application/vnd.api+json"})
resp.raise_for_status()
j = resp.json()

# 2) Grab the list under "data"
items = j["data"]

# 3) Normalize into a DataFrame
#    This yields columns including:
#      id, type,
#      attributes_gln, attributes_name, attributes_organisationNumber, attributes_partyTypes,
#      relationships_grid-area_data  (a list of dicts)
df = pd.json_normalize(items, sep="_")

# 4) Clean up column names
df.columns = (
    df.columns
      .str.replace("attributes_", "")
      .str.replace("partyTypes", "party_types")
)

# 5) Extract _all_ EIC codes into a list
df["eic"] = df["relationships_grid-area_data"].apply(
    lambda lst: [d["id"] for d in lst] if isinstance(lst, list) else []
)

# 6) Drop the raw relationships column
df = df.drop(columns=["relationships_grid-area_data"])

# 7) (Optional) Select/reorder the columns you care about
df = df[[
    "id",               # party ID (same as GLN)
    "gln",
    "name",
    "organisationNumber",
    "party_types",
    "eic"               # list of grid-area IDs now
]]

# 8) Inspect
df


,id,gln,name,organisationNumber,party_types,eic
0,7080003816870,7080003816870,Linja AS Regulert kraftleverandør,912631532,"[BalanceResponsible, RegulatedBalanceSupplier]",[]
1,7080003819864,7080003819864,Linja (tidl SFE) REG Kraftleverandør,984882114,[RegulatedBalanceSupplier],[]
2,7080003823953,7080003823953,Krødsherad Everk KF - Regulert kraftleverandør,971030658,[RegulatedBalanceSupplier],[]
3,7080005052382,7080005052382,Telenor Eiendom Holding As,971050365,[BalanceSupplier],[]
4,7080003819383,7080003819383,Barents Nett AS Pliktleveranse,971058854,[RegulatedBalanceSupplier],[]
...,...,...,...,...,...,...
881,7080003400673,7080003400673,NECS,962986633,[CertificateIssuer],[]
882,7080004073456,7080004073456,Statnett Forskningsprosjekt 2019-2021,962986633,[ThirdParty],[]
883,7080004066618,7080004066618,Nettselskapet AS reg. kraftlev,921688679,"[ServiceProvider, RegulatedBalanceSupplier]",[]
884,7080004066526,7080004066526,Nettselskapet AS - Sodvin-Regulert kraftlev,921688679,"[ServiceProvider, RegulatedBalanceSupplier]",[]


In [5]:
import requests
import pandas as pd

# 1) Fetch the JSON
url = "https://api.elhub.no/energy-data/v0/parties"
resp = requests.get(url, headers={"Accept": "application/vnd.api+json"})
resp.raise_for_status()
j = resp.json()

# 2) Grab the list under "data"
items = j["data"]

# 3) Normalize into a DataFrame
df = pd.json_normalize(items, sep="_")

# 4) Clean up column names
df.columns = (
    df.columns
      .str.replace("attributes_", "")
      .str.replace("partyTypes", "party_types")
)

# 5) Extract _all_ EIC codes into a list
df["eic"] = df["relationships_grid-area_data"].apply(
    lambda lst: [d["id"] for d in lst] if isinstance(lst, list) else []
)

# 6) Drop the raw relationships column
df = df.drop(columns=["relationships_grid-area_data"])

# 7) Select/reorder the columns you care about
df = df[[
    "id",               # party ID (same as GLN)
    "gln",
    "name",
    "organisationNumber",
    "party_types",
    "eic"               # list of grid-area IDs now
]]

# 8) Build a new DataFrame with only rows that have ≥1 EIC
df_with_eic = df[df["eic"].apply(lambda lst: len(lst) > 0)].reset_index(drop=True)

# 9) Inspect
print("All parties:", len(df))
print("With ≥1 EIC:", len(df_with_eic))
df_with_eic


All parties: 886
With ≥1 EIC: 130


,id,gln,name,organisationNumber,party_types,eic
0,7080005054928,7080005054928,Vestmar Nett AS,979399901,[GridOwner],[50Y-7FS806KF24QR]
1,7080005050616,7080005050616,VENI Metering AS,980405540,[GridOwner],"[50YPCGLIAF00FNY6, 50YQ9HSIFAQO5T0G, 50Y0461ER..."
2,7080005053259,7080005053259,"Yara Norge AS, Yara Glomfjord",984015666,[GridOwner],[50YBDMJ0PZ1-8GH3]
3,7080003380630,7080003380630,Ulefos Kraftverk DA,992840978,[GridOwner],[50YJH0AW0Y58EXYZ]
4,7080003823038,7080003823038,Norske Skog Skogn AS Nett,996732673,[GridOwner],[50YAJDAXAKFVSAM4]
...,...,...,...,...,...,...
125,7080001266455,7080001266455,Eramet Norway AS DSO,980518647,[GridOwner],[50Y6FR0SF1DC0L-D]
126,7080005050661,7080005050661,Lucerna AS,982897327,[GridOwner],[50YWZ9LMV1ZFT6HK]
127,7080005046466,7080005046466,Nordkraft Nett AS,983099807,[GridOwner],"[50Y1003TMTAXKZ1E, 50YCRBS1W9YF0AF5, 50YGBS9DP..."
128,7080004122741,7080004122741,SFE PRODUKSJON AS - NETTEIER,984882106,[GridOwner],[50YOEWOFS0MLA5M8]


In [6]:
# explode turns each list into multiple rows
df_exploded = df_with_eic.explode('eic')

# if you have some placeholder values you want to ignore,
# e.g. '*' or None, you can drop them:
df_exploded = df_exploded[~df_exploded['eic'].isin(['*', None])]

# now just ask pandas for the nunique()
n_unique = df_exploded['eic'].nunique()
print(f"Unique EICs: {n_unique}")


Unique EICs: 419


In [7]:
# --- assume you already have big_df and df_with_eic defined as above ---

# 1) Build a flat mapping from each EIC code → the corresponding owner name (or ID)
grid_owner_map = {
    code: row["name"]               # or row["id"] if you’d rather store the party’s GLN
    for _, row in df_with_eic.iterrows()
    for code in row["eic"]
}

# 2) Map big_df.eic through that dict, producing NaN where there’s no match
big_df["gridOwner"] = big_df["eic"].map(grid_owner_map)

# 3) (Optional) inspect
print(big_df[["eic", "gridOwner"]].drop_duplicates().head())


                  eic                     gridOwner
0    50Y-7FS806KF24QR               Vestmar Nett AS
168  50Y-AABJ1ZGH7DNR                     Sygnir AS
336  50Y-DNIYJ0PY0-73             Norefjell Nett AS
504  50Y013HNTAMLZ024            Føie AS - Hemsedal
672  50Y09NI57T7CCAVM  Indre Hordaland Kraftnett AS


In [8]:
big_df

,calculatedLossFraction,calculatedLossQuantityKwh,endTime,gridArea,lastUpdatedTime,netInfeedQuantityKwh,priceArea,startTime,eic,name,status,gridOwner
0,0.062064,1451.970,2023-01-01T01:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23394.820,NO2,2023-01-01T00:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active,Vestmar Nett AS
1,0.059934,1399.627,2023-01-01T02:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23352.914,NO2,2023-01-01T01:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active,Vestmar Nett AS
2,0.061977,1430.195,2023-01-01T03:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,23076.060,NO2,2023-01-01T02:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active,Vestmar Nett AS
3,0.060450,1373.440,2023-01-01T04:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,22720.318,NO2,2023-01-01T03:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active,Vestmar Nett AS
4,0.060322,1360.366,2023-01-01T05:00:00+01:00,KRAGERØ1,2025-03-28T22:03:12+01:00,22551.768,NO2,2023-01-01T04:00:00+01:00,50Y-7FS806KF24QR,KRAGERØ1,Active,Vestmar Nett AS
...,...,...,...,...,...,...,...,...,...,...,...,...
2917782,0.036774,1139.471,2025-05-31T20:00:00+02:00,NNAS ASKØY,2025-06-14T18:34:35+02:00,30985.395,NO5,2025-05-31T19:00:00+02:00,50YZSOZBH-PXFAUA,NNAS ASKØY,Active,Norgesnett AS
2917783,0.038655,1204.240,2025-05-31T21:00:00+02:00,NNAS ASKØY,2025-06-14T18:34:35+02:00,31153.254,NO5,2025-05-31T20:00:00+02:00,50YZSOZBH-PXFAUA,NNAS ASKØY,Active,Norgesnett AS
2917784,0.038478,1172.379,2025-05-31T22:00:00+02:00,NNAS ASKØY,2025-06-14T18:34:35+02:00,30468.773,NO5,2025-05-31T21:00:00+02:00,50YZSOZBH-PXFAUA,NNAS ASKØY,Active,Norgesnett AS
2917785,0.041818,1261.724,2025-05-31T23:00:00+02:00,NNAS ASKØY,2025-06-14T18:34:35+02:00,30171.854,NO5,2025-05-31T22:00:00+02:00,50YZSOZBH-PXFAUA,NNAS ASKØY,Active,Norgesnett AS


In [16]:
# count non-null uniques
num_unique_owners = big_df["gridOwner"].dropna().nunique()
print(f"There are {num_unique_owners} unique grid owners in big_df.")


There are 82 unique grid owners in big_df.


In [9]:
import pandas as pd

# 1. Save your DataFrame to CSV
# Replace big_df with your DataFrame variable
big_df.to_csv('big_df.csv', index=False)